In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=4da8f400052d2ed7bf17dd8c5dc6a3082d279447d449481fd83e120934d98707
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.3 GB  | Proc size: 158.9 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 23kB/s 


# Utility

In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
from keras import backend as K

class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename)
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['text_cleaned'], float):
                texts.append(str(row['text_cleaned']))
            else:
                texts.append(row['text_cleaned'])

            labels.append(row['target'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = tf.keras.preprocessing.sequence.pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_testing_metric(self, y_test, y_pred):
        accuracyScore = accuracy_score(y_test, y_pred)
        precisionScore = precision_score(y_test, y_pred)
        recallScore = recall_score(y_test, y_pred)
        f1Score = f1_score(y_test, y_pred)

        return accuracyScore, precisionScore, recallScore, f1Score

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_val_data(self, textsTraining, labelsTraining, train_index, val_index, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining)[train_index], np.asarray(textsTraining)[val_index]
        y_train, y_val = np.asarray(labelsTraining)[train_index], np.asarray(labelsTraining)[val_index]

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)

    def recall(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_score(self, y_true, y_pred):
        precision = self.precision(y_true, y_pred)
        recall = self.recall(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


# CNN

In [4]:
import tensorflow as tf

class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=False))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        if indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        util = utility()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[util.f1_score])

        return model

# Path

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

# path
dataTraining = 'trainPreprocessed.csv'
dataTesting = 'testPreprocessed.csv'
testing_name = "test"
root_path = 'gdrive/My Drive/Colab Notebooks/Text-Classification'
datasetPath = root_path + '/Disaster Tweets/Dataset/'
resultsPath = root_path + '/Disaster Tweets/Paper GA-CNN/Results/'
archPath = root_path + '/Disaster Tweets/Paper GA-CNN/Architecture/'
glovePath = [root_path + '/Glove/glove.6B.50d.txt',
             root_path + '/Glove/glove.6B.100d.txt',
             root_path + '/Glove/glove.6B.200d.txt',
             root_path + '/Glove/glove.6B.300d.txt']
bestParameters = root_path + '/Disaster Tweets/Paper GA-CNN/BestResults.csv'

KeyboardInterrupt: ignored

# Run

In [ ]:
import time
import os
import csv
import tensorflow as tf

# parameters
n_run = 30

# object class
util = utility()
cnn = CNN()

with open(bestParameters, mode='r') as infile:
    reader = csv.reader(infile)
    hyperparameters = {}
    num_columns = len(next(reader))
    infile.seek(0)
    for row in reader:
        if row[0] == 'layers':
            hyperparameters[row[0]] = [row[column] for column in range(1, num_columns)]
        else:
            if 'activation' in row[0] or 'optimizer' in row[0] or 'init' in row[0]:
                hyperparameters[row[0]] = row[1]
            elif 'rate' in row[0] or 'momentum' in row[0]:
                hyperparameters[row[0]] = float(row[1])
            else:
                hyperparameters[row[0]] = int(row[1])

# Read data
df = util.read_CSV(datasetPath + dataTraining)
dfTesting = util.read_CSV(datasetPath + dataTesting)

from sklearn.model_selection import train_test_split
dfTraining, dfVal = train_test_split(df, test_size = 0.3, random_state=42)

# get texts and labels
textsTraining, y_train = util.get_text_label(dfTraining)
textsVal, y_val = util.get_text_label(dfVal)
textsTesting, y_test = util.get_text_label(dfTesting)

# Tokenize words
tokenizer = util.tokenize_texts(textsTraining)
X_train = tokenizer.texts_to_sequences(textsTraining)
X_val = tokenizer.texts_to_sequences(textsVal)
X_test = tokenizer.texts_to_sequences(textsTesting)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# get maxlen
maxlen = util.get_max_length_of_sentences(textsTraining)

# Pad sequences with zeros
X_train = util.padding_texts(X_train, maxlen)
X_val = util.padding_texts(X_val, maxlen)
X_test = util.padding_texts(X_test, maxlen)

if int(hyperparameters['output_dim']) == 50:
    glove = glovePath[0]
elif int(hyperparameters['output_dim']) == 100:
    glove = glovePath[1]
elif int(hyperparameters['output_dim']) == 200:
    glove = glovePath[2]
elif int(hyperparameters['output_dim']) == 300:
    glove = glovePath[3]

embedding_matrix = util.create_embedding_matrix(glove, tokenizer.word_index, int(hyperparameters['output_dim']))

# Create Testing Results
f = open(resultsPath + testing_name + ".csv", "w+")
f.write("i,accuracy,precision,recall,f1Score,time\n")
f.close()

for i in range(0, n_run):
    then = time.time()
    # grid search
    model = cnn.cnn_model(vocab_size, maxlen, embedding_matrix, hyperparameters, hyperparameters['layers'])
    
    # save history to a file
    callbacks = [tf.keras.callbacks.CSVLogger(str(archPath + testing_name + str(i + 1) + ".csv"))]

    #early stopping
    callbacks += [tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', verbose=1, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(archPath + testing_name + str(i + 1) + ".h5", monitor='val_f1_score', mode='max', verbose=1, save_best_only=True)]
       
    model.summary()
    y_train = np.uint8(y_train)
    y_val = np.uint8(y_val)
    y_test = np.uint8(y_test)
    model.fit(X_train, y_train, epochs=hyperparameters['epochs'], verbose=True, validation_data=(X_val, y_val),
              batch_size=hyperparameters['batch_size'], callbacks=callbacks)
    
    # save model to a file
    # model.save(archPath + testing_name + str(i + 1) + ".h5")

    dependencies = {
    'f1_score': util.f1_score
    }

    # load the saved model
    saved_model = tf.keras.models.load_model(archPath + testing_name + str(i + 1) + ".h5", custom_objects=dependencies)   

    y_pred = saved_model.predict_classes(X_test)
    dfTesting['target'] = y_pred
    util.write_df_csv(dfTesting, resultsPath + testing_name + str(i + 1) + ".csv")
    

    # CNN metrics
    accuracyScore, precisionScore, recallScore, f1Score = util.get_testing_metric(y_test, y_pred)

    now = time.time()
    diff = now - then
    print(diff)
    print(f1Score)

    # save testing data
    f = open("{}{}.csv".format(resultsPath, testing_name), 'a')
    text = "{0},{1},{2},{3},{4},{5}\n".format(str(i + 1),str(accuracyScore), str(precisionScore), str(recallScore), str(f1Score), str(diff))
    f.write(text)
    f.close()